In [ ]:
import pandas as pd
import json

In [ ]:
df_final = pd.read_csv("C:/Users/Usuario/Documents/Adalab49/Módulo 3/Proyecto_módulo 3/Copia/project-da-promo-49-modulo-3-team-5/fase_2_transformacion/df_final.csv")


In [ ]:
df_final.head()

In [ ]:
df_final.columns

In [ ]:
df_final.columns = df_final.columns.str.strip()
#Para borrar espacios en blanco

In [ ]:
df_final = df_final.loc[:, ~df_final.columns.str.contains("^Unnamed")]
# Para borrar las columnas Unnamed

In [ ]:
df_final.fillna(value='', inplace=True) # Esto reemplaza NaN por None, que se convierte en NULL en MySQL.

In [ ]:
# Hay que dividirlo en 3 tablas porque ahora solo tenemos una que queremos convertir en 3
# Tabla 1: employee_details
df_employee_details = df_final[['employee_id', 'age', 'distancefromhome', 'education',
                                'educationfield', 'gender', 'maritalstatus', 'numcompaniesworked',
                                'totalworkingyears', 'datebirth']]

In [ ]:
# Tabla 2: employee_satisfaction
df_employee_satisfaction = df_final[['employee_id', 'environmentsatisfaction', 'jobsatisfaction',
                                     'relationshipsatisfaction', 'worklifebalance', 'jobinvolvement']]

In [ ]:
# Tabla 3: job_details
df_job_details = df_final[['employee_id', 'attrition', 'businesstravel', '$_dailyrate', 'department',
                           '$_hourlyrate', 'joblevel', 'jobrole', '$_monthlyincome', '$_monthlyrate',
                           'overtime', '%_percentsalaryhike', 'performancerating', 'standardhours',
                           'stockoptionlevel', 'trainingtimeslastyear', 'yearsatcompany',
                           'yearsincurrentrole', 'yearssincelastpromotion', 'yearswithcurrmanager',
                           '$_salary', 'remotework']]


In [ ]:
df_job_details.columns = df_job_details.columns.str.replace(r'[$%]', '', regex=True)
df_job_details.columns = df_job_details.columns.str.replace(r'^_', '', regex=True)
# Eliminar símbolos que dan error

In [ ]:
lista_details = df_employee_details.values.tolist()
lista_satisfaction = df_employee_satisfaction.values.tolist()
lista_job = df_job_details.values.tolist()

In [ ]:
print(df_job_details.columns.tolist())  


In [ ]:
import mysql.connector
from mysql.connector import errorcode

In [ ]:
cnx = mysql.connector.connect(user='root', password='AlumnaAdalab',
                              host='127.0.0.1',
                              database ='proyecto_abc')

mycursor = cnx.cursor() # creamos el "camioncito que transporta"

In [ ]:
# Consulta SQL para INTRODUCIR DATOS de employee_details
query = """
INSERT INTO employee_details (employee_id,age,distancefromhome,education,educationfield,gender,maritalstatus,numcompaniesworked,totalworkingyears,datebirth)
VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)
"""
mycursor.executemany(query, lista_details)

In [ ]:
# Consulta SQL para INTRODUCIR DATOS de employee_satisfaction

query2 = """
INSERT INTO employee_satisfaction (employee_id,environmentsatisfaction,jobsatisfaction,relationshipsatisfaction,worklifebalance,jobinvolvement)
VALUES (%s,%s,%s,%s,%s,%s)
"""
mycursor.executemany(query2, lista_satisfaction)

In [ ]:
# Consulta SQL para INTRODUCIR DATOS de job_details
query3 = """
INSERT INTO job_details (employee_id, attrition, businesstravel, dailyrate, department, hourlyrate, joblevel, jobrole, monthlyincome, monthlyrate, overtime, percentsalaryhike, performancerating, standardhours, stockoptionlevel, trainingtimeslastyear, yearsatcompany, yearsincurrentrole, yearssincelastpromotion, yearswithcurrmanager, salary, remotework)
VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)
"""

# Asegúrate de que lista_job tiene la misma cantidad de columnas que la consulta
mycursor.executemany(query3, lista_job)

In [ ]:
lista_de_tuplas = []
# Insertar datos
try:
    mycursor.executemany(query, lista_de_tuplas)
    cnx.commit()
    print("✅ Datos insertados correctamente.")
    print(mycursor.rowcount, "registros insertados.")
except mysql.connector.Error as err:
    print("❌ Error al insertar:", err)
    cnx.rollback()

# Cerrar conexión
mycursor.close()
cnx.close()